<a href="https://colab.research.google.com/github/gnishtha/Briqs/blob/master/fullworking_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install google-cloud-vision

     |████████████████████████████████| 440kB 2.7MB/s 


In [2]:
!apt-get install poppler-utils 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 59 not upgraded.
Need to get 154 kB of archives.
After this operation, 613 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 poppler-utils amd64 0.62.0-2ubuntu2.10 [154 kB]
Fetched 154 kB in 1s (276 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 144328 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.62.0-2ubuntu2.10_amd64.deb ...
Unpacking poppler-utils (0.62.0-2ubuntu2.10) ...
Setting up poppler-utils (0.62.0-2ubuntu2.10) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


In [6]:
!pip install pdf2image

In [4]:
#data science codes starts here

import io
import os
from google.cloud import vision
from google.cloud.vision import types
import pandas as pd
import json
import cv2
import matplotlib.pyplot as plt
from collections import Counter
from dateutil import parser
import re
from google.cloud import storage
from google.protobuf import json_format
from requests.utils import requote_uri
from urllib.request import urlopen
from PIL import Image

In [65]:
from pdf2image import convert_from_bytes
from google.cloud import storage
import io
import os
from requests.utils import requote_uri
from urllib.request import urlopen


# pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=r'/content/drive/My Drive/briq-chronicle-8e5681b27d06.json' 
# to test the application use below script
# print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])
client = storage.Client()
# C:\Users\Tarun Saini\Desktop\briqtest\poppler-0.68.0_x86\poppler-0.68.0\bin
# The os.eviron method returns a dict object of the users PATH



#this function is to get the add the poppler bin path for the environment 
# def setospath():
#     # C:\Users\Tarun Saini\Desktop\briqtest\poppler-0.68.0_x86\poppler-0.68.0\bin
#     # The os.eviron method returns a dict object of the users PATH
#     path = os.environ['PATH']
#     path = path + r';C:\Users\Tarun Saini\Desktop\briqtest\poppler-0.68.0_x86\poppler-0.68.0\bin' # Append the path to bin as a string
#     os.environ['PATH'] = path # Override value of 'PATH' key in the dict
# #     print(os.environ['PATH']) # This is the new updated PATH
# #     os.system('tarun') # Using system shell to call a program that was not on my PATH and now is
#     return print('success')


def getprefix(foldername,filename):
    if filename == '':
        customprefix= foldername+'/'
    else:
        customprefix= foldername + '/'+ filename
    return customprefix


def getfileurllisttoprocess(bucketname,foldername, filename = '', filetype= 'pdf'):
    # how to get all files as list
    client = storage.Client()
    fileurllisttoprocess =[]
    input_path_detail =[]
    delimiter=  '/'
    for blobnames in client.list_blobs(bucketname,delimiter = delimiter, prefix= getprefix(foldername,filename) ):
        if (blobnames.name.lower().endswith(filetype)):
#             print(blobnames.name)
            input_path_url = r'https://storage.cloud.google.com/'+bucketname+'/' +blobnames.name
            input_path_url = requote_uri(input_path_url)
            input_path_detail.append([input_path_url,bucketname,blobnames.name, blobnames.name.split("/")[-1]])
            fileurllisttoprocess.append(input_path_detail)
    return fileurllisttoprocess

def getbufferofblog(bucketname,blobfile):
    #print(blobfile)
    bucket = client.get_bucket(bucketname)
    blob = bucket.get_blob(blobfile)
    #print(blob)
    #print(blob.download_as_string())
    return blob.download_as_string()


def upload_to_bucket(imagefile,name , bucketname, imagefoldername):
    bucket = client.get_bucket(bucketname)
    blob = bucket.blob(getprefix(imagefoldername,name))
    with io.BytesIO() as output:
        imagefile.save(output, format="JPEG")
        contents = output.getvalue()
        blob.upload_from_string(contents)
#         blob.upload_from_string(imagefile.tobytes())
    return print("i worked")


def fetchpdfandimagecreation(vendorid, bucketname , foldername,imagefoldername='invoice_data', filename=''):
    #setospath()
    listprocess =[]
    urllist = getfileurllisttoprocess(bucketname,foldername, filename)
    for eachpdffile in urllist:
        inmemoryfile = getbufferofblog(eachpdffile[0][1],eachpdffile[0][2])        
        images = convert_from_bytes(inmemoryfile)
        for index,eachimage in enumerate(images):
            imagename = str(vendorid)+"__"+eachpdffile[0][2].split('/')[-1] +'__'+str(index) +'.jpg'
#             eachimage.save('a.jpg')
            upload_to_bucket( eachimage,imagename,bucketname,imagefoldername)
            listprocess.append([vendorid,eachpdffile[0][3],imagefoldername,imagename])
    return listprocess


In [66]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [75]:
"""
input -> folder path in which images and config yaml file is stored
output-> will extract all info's of field that are mention in yaml file
"""
def get_invoice_data_v1(configjson, image):
    # if imagefiles==None:
    #   files = list(set(os.listdir(path))-set([jsonfile]))
    # else:
    #   files = [path+imagefile for imagefile in imagefiles]
    #print(files)
    data = []
    t=[]
    # for file in files:
    mapping = configjson
    #print(mapping)
    temp_df = detect_text_v1(image)
    #print(temp_df)
    tmp_dict = {}
    inTableMapping={}
    for k in mapping.keys():
        #print(k)
        if mapping[k]['isTable'] == 'True' and  mapping[k]['moveable'] == 'False' :
            #print(k)
            mapping[k].pop('moveable')
            mapping[k].pop('isTable')
            tmp_dict[k] = get_table_structure_data(get_sequence_data(temp_df),mapping[k],False)
        elif mapping[k]['isTable'] == 'True' and  mapping[k]['moveable'] == 'True' :
            mapping[k].pop('moveable')
            mapping[k].pop('isTable')
            tmp_dict[k] = get_table_structure_data(get_sequence_data(temp_df),mapping[k],True)
        elif mapping[k]['isTable'] == 'False' and  mapping[k]['moveable'] == 'False' :
            #print(k)
            #mapping[k].pop('moveable')
            #mapping[k].pop('isTable')
            filter_data = temp_df[(mapping[k]['x0'] < temp_df['centerx']) & (temp_df['centerx'] < mapping[k]['x1']) & (mapping[k]['y0'] < temp_df['centery']) & (temp_df['centery'] < mapping[k]['y1'])].sort_values(by='x0')
            tmp_dict[k] = data_format((" ").join(get_sequence_data(filter_data)['text']),mapping[k]["type"])
        elif mapping[k]['isTable'] == 'False' and  mapping[k]['moveable'] == 'True' :
            #print(k)
            tmp_dict[k] = get_moveable_field_data(get_sequence_data(temp_df),mapping[k],False)
        #print(tmp_dict)
    data.append(tmp_dict)
    return data

In [9]:
def detect_text_v1(content):
    """Detects text in the file. Using google vision api (output will be in dataframe format)"""
    client = vision.ImageAnnotatorClient()

    # with io.open(path, 'rb') as image_file:
    #     content = image_file.read()

    image = vision.types.Image(content=content)

    response = client.text_detection(image=image)
    texts = response.text_annotations
    txt = []
    x0 = []
    x1 = []
    y0 = []
    y1 = []
    centerx = []
    centery = []
    for text in texts[1:]:
        #print('\n"{}"'.format(text.description))
        txt.append(text.description)
        vertices = ([(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])
        x0.append(vertices[0][0]) 
        x1.append(vertices[1][0])
        y0.append(vertices[0][1])
        y1.append(vertices[2][1])
        centerx.append(int((vertices[0][0]+vertices[1][0])/2))
        centery.append(int((vertices[0][1]+vertices[2][1])/2))
        #print('bounds: ',(vertices))
    return pd.DataFrame({"text":txt,"x0":x0,"y0":y0,"x1":x1,"y1":y1,"centerx":centerx,"centery":centery})
    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [10]:
"""Take dataframe as input and add column line for each words  """
def get_sequence_data(data):
    df = data.sort_values(by = ["centery"])
    lst = list(df['centery'])
    count = 0
    line = []
    for i in range(len(lst)):
        if i != len(lst)-1:
            if lst[i+1] - lst[i] > 8:
                line.append(count)
                count += 1
            else:
                line.append(count)
        else:
            line.append(count)
    df['line'] = line
    df = df.sort_values(by=['line','x0'])
    df.reset_index(drop=True, inplace=True)
    #print("xxx")
    #print(df)
    return df

In [11]:
"""
input -> start_text of and end_text of field
output -> return info of field (for moveable data)
"""
def get_moveable_field_data(data,key,isTable):
    start_data = data[data['text'].astype(str).str.contains(key['start'], case=False)]
    end_data = data[data['text'].astype(str).str.contains(key['end'], case=False)]
    temp_dct = {}
    for ind, row in start_data.iterrows():
        temp_dct[str(ind)+"_"+str(abs(end_data['line'] - row['line']).idxmin())] = min(abs(end_data['line'] - row['line']))
    indxs = min(temp_dct, key=temp_dct.get)
    strt = int(indxs.split("_")[0])
    end = int(indxs.split("_")[1])
    end_line = data.loc[end,'line']
    strt_line = data.loc[strt,'line']
    if isTable==False:
        if end_line != strt_line:
            return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (strt_line == data['line'])]['text']),key['type'])
        else:
            return data_format((" ").join(data[(data.loc[strt,'x1'] < data['x0']) & (data.loc[end,'x0'] > data['x1'])  & (strt_line == data['line'])]['text']),key["type"])
    # else:
    #     #print(key)
    #     return get_table_structure_data(data[(end_line > data['line']) & (strt_line <= data['line'])],key,True)

In [12]:
"""
Convert the types of info field as given in yaml file
"""
def data_format(x,frmt):
    try:
        if frmt == "float":
            return float(x)
        elif frmt == "int":
            return int(x) 
        elif frmt == 'date':
            return parser.parse(x).isoformat()
        else:
            return str(x)  
    except:
        return str(x)

In [13]:
"""
return key which has particular value (used in Pipline_for_get_complete_row function)
"""
def get_key(my_dict,val): 
    for key, value in my_dict.items(): 
        if val == value: 
            return key 

In [14]:
"""
This function is use to convert fetched table as given in images 
"""
def get_complete_row(df,key):
    restructure_data = {new_list: [] for new_list in df.columns}
    for index,row in df.iterrows():
        try:
            if row[key] != "":
                for i in df.columns:
                    restructure_data[i].append(row[i])
            else:
                for i in df.columns:
                    restructure_data[i] = restructure_data[i][:-1] + [restructure_data[i][-1] + row[i]]
        except:
            pass
    return restructure_data

In [15]:
"""
input -> raw table data (unformatted)
output -> same format tabe as in image
function use-> get_key(), get_complete_row()
"""
def Pipline_for_get_complete_row(table_data):
    df = pd.DataFrame(table_data)
    tmp = {new_list: len(list(filter(lambda x: x != "", df[new_list]))) for new_list in df.columns}
    val = Counter(list(tmp.values())).most_common(1)[0][0]
    key = get_key(tmp,val)
    updated_table_data = get_complete_row(df,key)
    return updated_table_data

In [16]:
"""
This function takes google vision data, and extract the table data (unformatted)
"""
def get_table_structure_data(data,key,moveable_flag):
    if moveable_flag == True:
        df = data
    else:
        #print(data)
        index = data.loc[data[data['text'].astype(str).str.contains(key['end'], case=False) & (data['y0'] > key['y0'])].index.values[0],'line']
        df = data[(key['x0'] < data['centerx']) & (data['centerx'] < key['x1']) & (key['y0'] < data['centery']) & (data['line'] < index)]
    #print(df,"-->",data[data['text'].str.contains(key['end'], case=False) & (data['y0'] > key['y0'])],index)
    key_list = list(set(list(key.keys()))-set(['x0','x1','y0','end','start']))
    table_data = {new_list: [] for new_list in key_list}
    #print(df) 
    for i in key_list:
        #print(i)
        flag = False
        for k in list(set(list(df['line']))):
            temp_df = df[(df['line']==k) & (key[i]['x0'] < df['centerx']) & (key[i]['x1'] > df['centerx'])]
            if flag:
                #print(key[i]["type"])
                table_data[i].append((" ").join(temp_df['text']))
            flag = True
    structure_table = Pipline_for_get_complete_row(table_data)
    type_mapping = {i:key[i]['type'] for i in key.keys() if type(key[i]) == dict}
    updated_structure_table = {i:[data_format(x,type_mapping[i]) for x in structure_table[i]] for i in structure_table.keys()}
    return updated_structure_table

In [81]:
#these are the inputs
#lets start with the code of image processing with frebase functions
#this data will come form the fire base

mapping= {
  "Address": {
    "x0": 230,
    "x1": 820,
    "y0": 100,
    "y1": 240,
    "moveable": "False",
    "isTable": "False",
    "type": "str"
  },
  "Invoice Number": {
    "x0": 1440,
    "x1": 1650,
    "y0": 230,
    "y1": 300,
    "moveable": "False",
    "isTable": "False",
    "type": "int"
  },
  "Invoive Date": {
    "x0": 1230,
    "x1": 1440,
    "y0": 230,
    "y1": 300,
    "moveable": "False",
    "isTable": "False",
    "type": "date"
  },
  "BILL to": {
    "x0": 150,
    "x1": 830,
    "y0": 430,
    "y1": 700,
    "moveable": "False",
    "isTable": "False",
    "type": "str"
  },
  "job site": {
    "x0": 910,
    "x1": 1600,
    "y0": 430,
    "y1": 700,
    "moveable": "False",
    "isTable": "False",
    "type": "str"
  },
  "total": {
    "x0": 1340,
    "x1": 1620,
    "y0": 970,
    "y1": 2030,
    "moveable": "False",
    "isTable": "False",
    "type": "int"
  },
  "table": {
    "x0": 100,
    "x1": 1630,
    "y0": 880,
    "end": "total",
    "moveable": "False",
    "isTable": "True",
    "date": {
      "x0": 100,
      "x1": 300,
      "type": "date"
    },
    "Ticket": {
      "x0": 300,
      "x1": 420,
      "type": "str"
    },
    "quantity": {
      "x0": 420,
      "x1": 545,
      "type": "int"
    },
    "comments": {
      "x0": 545,
      "x1": 975,
      "type": "str"
    },
    "description": {
      "x0": 975,
      "x1": 1300,
      "type": "str"
    },
    "price each": {
      "x0": 1300,
      "x1": 1470,
      "type": "int"
    },
    "amount": {
      "x0": 1470,
      "x1": 1630,
      "type": "int"
    }
  }
}
firebaseoutput = { 'is_process':'completed' ,"bounds": mapping}


#once all images are converted 
# vendorid, bucketname , foldername,imagefoldername, filename
vendorid1 = 12
bucketname1 = 'auto-invoicing-dev'
foldername1 = 'invoice_test'
imagefoldername1='invoice_data'
filename1 = 'Inv_38449_from_Excel_Disposal_LLC_8496.pdf'

In [82]:
def mainfunction(vendorid,bucketname,foldername,imagefoldername,filename, firebaseoutput):
    #request for getting images generated and uplaoded into the google storage
    #then getting the list of all images uploaded in the request
    listofimagestobeprocessed= fetchpdfandimagecreation(vendorid, bucketname , foldername,imagefoldername, filename)
    for eachimage in listofimagestobeprocessed:
        #call the cofig file json from firebase and save as firebaseoutput json based on image
        #firebaseoutput= functionname(imput vendor id hitting firebase index of google bounds)
#         print(getprefix(eachimage[2],eachimage[3]))
        imageasstring = getbufferofblog(bucketname, getprefix(eachimage[2],eachimage[3]))
        #return len(imageasstring)
#         get_invoice_data_v1(firebaseoutput['bounds'],imageasstring)  

    return print(get_invoice_data_v1(firebaseoutput['bounds'],imageasstring))
# mainfunction("12",bucketname1,foldername1,imagefoldername1,filename1)

In [83]:
mainfunction("12",bucketname1,foldername1,imagefoldername1,filename1, firebaseoutput)
#bucketname is the parent bucket where all folders are located
#foldername1 is the folder location where our file is present in the google bucket
#imagefoldername1 is the folder where code will push the converted images. default : invoice_data
#filename1 is optional if leaved as "" blank it will get all pdf located in the folder and fect
#firebaseoutput  variable need to be fetched from the "google_bounds" index 
#output of the main function will be passed to firebase "google extracted text"

i worked
[{'Address': 'Excel Disposal, LLC 5817 46th St. Kenosha, WI 53144', 'Invoice Number': 38449, 'Invoive Date': '2019-09-11T00:00:00', 'BILL to': 'Berglund Construction 8410 S. South Chicago Ave Chicago, IL 60617', 'job site': '5001 Simmons Island rd Kenosha, WI', 'total': '20.00 20.00 $390.00', 'table': {'comments': ['', ''], 'date': ['2019-09-05T00:00:00', ''], 'amount': ['370.00', '20.00'], 'Ticket': ['', ''], 'quantity': [1, 1], 'description': ['12yd GD Delivery', 'Dot Regulations Fee'], 'price each': ['370.00', '20.00']}}]


In [ ]:
print('''json output of the get_invoice_data_v1(firebaseoutput['bounds'],imageasstring) is coming 3 time where as it was only called once''')

json output of the get_invoice_data_v1(firebaseoutput['bounds'],imageasstring) is coming 3 time where as it was only called once
